## Επεξεργασία Φωνής και Φυσικής Γλώσσας 
### 1η Προπαρασκευαστική Εργασία 
### Θεδωρόπουλος Νικήτας -03115185
### Καλλιώρα Δωροθέα - 03115176 ? 




Σκοπός της εργαστηριακής άσκησης είναι η δημιουργία ενός απλού ορθογράφου με χρήση μηχανών πεπερασμένων καταστάσεων (fst) της βιβλιοθήκης openfst (v1.6.1). Θα χρησιμοποιήσουμε corpus απο δημόσια διαθέσιμα βιβλία για να δημιουργήσουμε ενα γλωσσικό μοντέλο σε μορφή λεξικού. Τέλος θα γίνει εισαγωγή σε αναπαραστάσεις word2vec. 

Κατασκευάζουμε το corpus με σύμπτηξη plain text βιβλίων, δημόσια διαθέσημα στο Project Gutenberg. Θα χρησιμοποιήσουμε corpus 2 βιβλίων The War of the Worlds by H. G. Wells και Grimms' Fairy Tales by Jacob Grimm and Wilhelm Grimm. 

Η τεχνικής της συνένωσης πολλών βιβλίων, εκτός απο το προφανές πλεονέκτημα του μεγαλύτερου train corpus, αυξάνει την αναγνωριστική ισχύ του μοντέλου αφού μπορεί να αντλήσει λεξιλόγειο απο διαφορετικές πηγές και άρα να έχει μεγαλύτερο corpus λέξεων.Ακόμα μειώνει το bias που προκύπτει απο ενα συγκεκριμένο βιβλίο ή απο έναν συγκεκριμένο συγγραφέα, που χρησιμοποιεί συγκεκριμένες λέξεις λόγω προτίμησης, εποχής συγγραφής και άλλων παραγόντων. 


In [66]:
#!wget -P ./data/ http://www.gutenberg.org/files/36/36-0.txt
#!wget -P ./data/ http://www.gutenberg.org/files/2591/2591-0.txt

In [113]:
# Step 2

def identity_preprocess(str):
  return str

def readfile(path,preprocess=identity_preprocess):
  processed_txt=[]
  f = open(path, "r")
  for line in f:
    processed_txt= processed_txt + preprocess(line)
  return processed_txt 

def tokenize(s):
    s = s.strip().lower()
    s = ''.join(c if c.isalpha() else ' ' for c in s)
    # We replace all non-alpha characters with ' '
    s = s.replace('\n',' ')
    s = s.split()
    return s
  
# TODO: experiment with ntk tokenizers 




In [114]:
# Step 3

text_1 = readfile('./data/36-0.txt',tokenize)
text_2 = readfile('./data/2591-0.txt',tokenize)
text = text_1 + text_2
print(len(text),"words overall")
word_corpus = list(set(text))
print(len(word_corpus),"unique words in corpus")
# Convert list of words to list of chars, get unique chars with set
alphabet = list(set([c for word in word_corpus for c in word]))
print(len(alphabet),"symbols in alphabet:")
print(alphabet)




64183 words overall
7072 unique words in corpus
28 symbols in alphabet:
['æ', 'n', 'i', 'q', 'e', 'j', 'k', 'o', 'f', 'w', 'd', 'x', 'u', 'z', 'v', 'b', 't', 'r', 'm', 'ç', 'g', 's', 'y', 'p', 'h', 'a', 'l', 'c']


Με βάση το αλφάβητο που προέκυψε απο το word corpus δημιουργούμε το αρχείο συμβόλων εισόδου και εξόδου για τα fst.

In [115]:
# Step 4

def create_syms(alphabet):
    f = open("./chars.syms","w+")
    f.write(f'<epsilon>     0\n')  #for <epsilon> index 0
    for i in range(len(alphabet)):
        f.write(f'{alphabet[i]}     {i+50}\n')  #for other characters index i+50
        
create_syms(alphabet)
    

Ως μετρική για την απόσταση 2 λέξεων θα χρησιμοποιήσουμε την απόσταση Levenshtein (ή edit distance). Η απόσταση μπορέι να υπολογιστεί αναδρομικά με χρήση dynamic programming (DP). Ανάμεσα σε δύο λέξεις μπορούν να γίνουν τρείς τύποι μετατροπών: εισαγωγή ενός χαρακτήρα ($ \epsilon \rightarrow a$), μετατροπή ενός χαρακτήρα ($ a \rightarrow b$) και διαγραφή ενός χαρακτήρα ($a \rightarrow \epsilon$). Θεωρούμε αρχικά ίδιο κόστος για όλες τις μετατροπές.

Δημιουργούμε κατάλληλο fst με μία κατάσταση, αντιστοιχίζοντας κάθε χαρακτήρα σε κάθε άλλον και στο $\epsilon$, και το $\epsilon$ σε κάθε χαρακτήρα με βάρος 1. Αντιστοιχίζουμε ακόμα τον χαρακτήρα στον εαυτό του με βάρος 0. Αν πάρουμε το shortest path τότε η έξοδος θα είναι λέξη εισόδου, αφού το ελάχιστο βάρος προκύπτει αν δεν γίνει καμία μετατροπή. 

β) ????

In [116]:
#Step 5

# arc format: src dest ilabel olabel [weight]
# final state format: state [weight]
# lines may occur in any order except initial state must be first line
# unspecified weights default to 0.0 (for the library-default Weight type) 

f = open('lev.fst',"w+")

def format_arc(src,dst,src_sym,dst_sym,w,f,acceptor=False):
    if not acceptor:
        f.write("{} {} {} {} {} \n".format(src,dst,src_sym,dst_sym,w))
    else:
        f.write("{} {} {} \n".format(src,dst,src_sym))
letters =  alphabet

for i in range(0, len(letters)):
    format_arc(src=0, dst=0, src_sym="<epsilon>", dst_sym=letters[i], w=1,f=f)
    format_arc(src=0, dst=0, src_sym=letters[i], dst_sym=letters[i], w=0,f=f)
    format_arc(src=0, dst=0, src_sym=letters[i], dst_sym="<epsilon>", w=1,f=f)
    for j in range(0, len(letters)):
        if(j!=i):
            format_arc(src=0, dst=0, src_sym=letters[i], dst_sym=letters[j], w=1,f=f)   
f.write('0\n')
f.close()
    

#TODO: change the edit distance

In [117]:
!fstcompile --isymbols=chars.syms --osymbols=chars.syms  lev.fst lev.bin.fst
!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait lev.bin.fst | dot -Tjpg >lev.jpg

Ένας αποδοχέας είναι ένα fst όπου κάθε μετάβαση έχει ενα label (και προεραιτικά βάρος). Στην βιβλιοθήκη fst μπορέι να θεωρηθεί ως μετατροπέας με ίδιο input και output label. Κατασκευάζουμε εναν αποδοχέα που αποδέχεται κάθε λέξη του λεξικού.

In [136]:
#Step 6


def create_acceptor(words,file):
    f = open(file,"w+")
    s = 0
    final_states = []
    for word in words:
        format_arc(0,s+1,word[0],word[0],0,f,acceptor=True)
        s += 1
        for letter in word[1:]:
            format_arc(s,s+1,letter, letter, 0,f,acceptor=True)
            s += 1
        final_states.append(s)

    for state in final_states:
        f.write(f'{state}\n')
    f.close()    
create_acceptor(word_corpus,"acceptor.fst")

In [137]:
!fstcompile --isymbols=chars.syms --osymbols=chars.syms --acceptor acceptor.fst acceptor.bin.fst
#!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait rosebud.bin1.fst | dot -Tjpg >acceptor.jpg

#TODO: Τι κάνουν οι απο κατω συναρτήσεις απο την βιβλιοθήκη..... 

In [138]:
#Step 6b

!fstrmepsilon acceptor.bin.fst acceptor.bin.fst
#!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait acceptor.bin.fst | dot -Tjpg >dfa_rmepsilon.jpg

!fstdeterminize acceptor.bin.fst acceptor.bin.fst
#!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait acceptor.bin.fst | dot -Tjpg >dfa_determinize.jpg

!fstminimize acceptor.bin.fst acceptor.bin.fst
#!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait acceptor.bin.fst | dot -Tjpg >dfa_minimize.jpg


In [139]:
#Step 7
#mini.fst - lev.fst

!fstarcsort --sort_type=olabel lev.bin.fst lev.bin.fst
!fstarcsort --sort_type=ilabel acceptor.bin.fst acceptor.bin.fst

In [140]:
!fstcompose  lev.bin.fst acceptor.bin.fst spell_checker.bin.fst
#!fstdraw --isymbols=chars.syms --osymbols=chars.syms -portrait spell_checker.bin.fst | dot -Tjpg >com.jpg

In [158]:
create_acceptor(['cit'],"in.fst")

In [159]:
!fstcompile  --isymbols=chars.syms --osymbols=chars.syms  --acceptor in.fst in.bin.fst

In [160]:
!fstarcsort --sort_type=ilabel spell_checker.bin.fst spell_checker.bin.fst 
!fstarcsort --sort_type=olabel in.bin.fst in.bin.fst 

In [161]:
!fstcompose in.bin.fst spell_checker.bin.fst |fstshortestpath --nshortest=1 \
| fstrmepsilon |  fsttopsort |fstprint -isymbols=chars.syms  -osymbols=chars.syms\
| cut -f4 | grep -v "<epsilon>" |head -n -1 | tr -d '\n'


cut

In [162]:
!fstcompose in.bin.fst spell_checker.bin.fst |fstshortestpath --nshortest=1   \
| fstrmepsilon |  fsttopsort |fstprint -isymbols=chars.syms  -osymbols=chars.syms

0	1	c	c
1	2	i	u	1
2	3	t	t
3


In [163]:
#!wget https://raw.githubusercontent.com/georgepar/python-lab/master/spell_checker_test_set -P ./data/

In [175]:
# Step 8:
import random
random.seed()
file = open('./data/spell_checker_test_set',"r")
y_test = []
X_test = []
for line in file:
    [y,X] = line.split(':')
    X = X.split()
    for word in X:
        X_test.append(word)
        y_test.append(y)

print(y_test[0:4])
print(X_test[0:4])
print(len(y_test))
print(len(X_test))
idxs = random.sample(range(0, len(y_test)), 20)
print(idxs)


['contented', 'contented', 'contented', 'contented']
['contenpted', 'contende', 'contended', 'contentid']
270
270
[133, 34, 43, 23, 138, 127, 260, 247, 29, 81, 221, 251, 188, 214, 130, 76, 1, 192, 110, 217]


In [176]:
def predict(Y,X):
    correct_pred=0
    for y,x in zip(Y,X):
        create_acceptor([x],"input.fst")

        !fstcompile  --isymbols=chars.syms --osymbols=chars.syms  --acceptor input.fst input.bin.fst
        !fstarcsort --sort_type=ilabel spell_checker.bin.fst spell_checker.bin.fst 
        !fstarcsort --sort_type=olabel input.bin.fst input.bin.fst 
        prediction = !fstcompose input.bin.fst spell_checker.bin.fst |fstshortestpath --nshortest=1 \
        | fstrmepsilon |  fsttopsort |fstprint -isymbols=chars.syms  -osymbols=chars.syms\
        | cut -f4 | grep -v "<epsilon>" |head -n -1 | tr -d '\n' 
        print("Wrong = ",x,"-- Correct = ",y,"-- Prediction = ",prediction[0])
        if y == prediction[0]:
            correct_pred+=1
    print(f"Accuracy:{correct_pred/len(Y)}%")


In [177]:
X_rand = [X_test[i] for i in idxs]
Y_rand = [y_test[i] for i in idxs]
predict(Y_rand,X_rand)

Wrong =  protend -- Correct =  pretend -- Prediction =  pretend
Wrong =  unexpcted -- Correct =  unexpected -- Prediction =  expected
Wrong =  biscuts -- Correct =  biscuits -- Prediction =  biscuits
Wrong =  independant -- Correct =  independent -- Prediction =  independent
Wrong =  wote -- Correct =  wrote -- Prediction =  note
Wrong =  chaphter -- Correct =  chapter -- Prediction =  chapter
Wrong =  adres -- Correct =  address -- Prediction =  agree
Wrong =  oppisit -- Correct =  opposite -- Prediction =  opposite
Wrong =  poety -- Correct =  poetry -- Prediction =  piety
Wrong =  totaly -- Correct =  totally -- Prediction =  coaly
Wrong =  failes -- Correct =  fails -- Prediction =  files
Wrong =  oppossitte -- Correct =  opposite -- Prediction =  opposite
Wrong =  planed -- Correct =  planned -- Prediction =  plane
Wrong =  embaras -- Correct =  embarrass -- Prediction =  mars
Wrong =  vairious -- Correct =  various -- Prediction =  various
Wrong =  arnt -- Correct =  aunt -- Pred